In [1]:
import PyPDF2
import re
import pandas as pd

In [2]:
def customer_details(filepath):
    
    text =''
    
    with open(filepath,'rb') as pdf:
        pdf_reader = PyPDF2.PdfReader(pdf)
        for pages in pdf_reader.pages:
            text += pages.extract_text()
    
    pdf_patterns = {'Account_number':r'Account\s+No\s+:(\d+)',
               'Customer ID':r'Customer\s+ID\s+:+(\d+)',
                'IFSC':r'IFSC\s+Code\s+:+([A-Za-z\s]+(\d+))',
                 'MICR_Code': r'MICR\s+Code\s+:+(\d+)',
                 'NoMinee Registered': r'Nominee\s+Registered\s+:\s+(Y|N)',
                 'PAN': r'PAN\s+:([A-Za-z0-9\s]+)',
                'Account_Type': r'Scheme\s+:([A-Za-z\s]+-[A-Za-z]+)',
                'Statement Period':r'period\s+.+?From\s+:\s+(\d{2}-\d{2}-\d{4})\s+To\s+:\s+(\d{2}-\d{2}-\d{4})'}
    
    account_name  = text.splitlines()[1]
    
    ## Customer Details
    
    customer_details = {}
    customer_details['Customer_name'] = account_name
    for name,pattern in pdf_patterns.items():
        
        match= re.search(pattern,text)
        
        if name in ('account_number','Customer/CIF_ID','Mobile_Number') :
            
            customer_details[name] = int(match.group(1))
        if name == 'Statement Period':
            customer_details[name] = match.group(1)+' '+'To'+' '+ match.group(2)
             
            
        else:
            
            customer_details[name] = match.group(1)
        
            

    
    return  customer_details,text

In [3]:
details,text = customer_details('Account_stmt_XX9588_09022024.pdf')

In [4]:
lines = text.replace('Init.\nBr', 'Init. Br').splitlines()

joinlines =[]
   
transactions = []
index = 0
a = 0
b = 0
header_found = False
for line in lines:
        if "Tran Date Chq No Particulars Debit Credit Balance Init. Br" in line:
            index = lines.index("Tran Date Chq No Particulars Debit Credit Balance Init. Br")
            i = index +2
            j = (len(lines)-index)
            
while i <= j:
    line = ''
    ts_line = lines[i].strip()
    if re.search(r'(\d{2}-\d{2}-\d{4})',ts_line):
        ws_line = lines[i+1].strip()
        if i+1 <len(lines) and not re.match(r'\d{2}-\d{2}-\d{4}', ws_line):
            line = ts_line+' '+ws_line
            i = i+1
            joinlines.append(line)
    else:
        joinlines.append(ts_line)
    i = i+1
                ## j +=1
                    
Transations = "\n".join(joinlines)
##print(Transations)           
            


In [5]:
line = Transations.splitlines()

In [6]:
Transations=[]
transations={}
i=1
for i in range(0,len(line)):
    a= line[i]
    ##print(a)
    match = re.match(r'(\d{2}-\d{2}-\d{4})(\S+)(.+?\s{2,})(.+?\s{2,})(.+?\d+.\d{2})(.+?\d+.\d{2})(.+?\d+)',a)
    if match:
        transations = {
        'Date':match.group(1).strip(),
        'Particulars':match.group(2).strip(),
        'Debit':match.group(4).strip(),
        'Credit':match.group(5).strip(),
        'Balance':match.group(6).strip(),
        'int.Br':match.group(7).strip(),
        }
        
        Transations.append(transations)
    else:
        pass
    
##print(Transations)

In [7]:
axis_bank_statement = []

In [8]:
details['Transactions'] = Transations

In [9]:
axis_bank_statement.append(details)

In [10]:
axis_bank_statement

[{'Customer_name': 'NILAM MAHENDRA BANGAR',
  'Account_number': '921010029229588',
  'Customer ID': '907410280',
  'IFSC': 'UTIB0003143',
  'MICR_Code': '411211051',
  'NoMinee Registered': 'Y',
  'PAN': 'AWBPB7206J\nScheme ',
  'Account_Type': 'SAVINGS-EASYACCESS',
  'Statement Period': '01-04-2023 To 31-03-2024',
  'Transactions': [{'Date': '26-05-2023',
    'Particulars': 'IMPS/P2A/314602490381/918020110872063/260',
    'Debit': '',
    'Credit': '52.00',
    'Balance': '52.00',
    'int.Br': '100'},
   {'Date': '20-07-2023',
    'Particulars': 'UPI/P2A/320131458783/BANGAR',
    'Debit': '',
    'Credit': '1.00',
    'Balance': '1.00',
    'int.Br': '3143'},
   {'Date': '29-07-2023',
    'Particulars': 'IMPS/P2A/321012358631/918020110872063/290',
    'Debit': '',
    'Credit': '2.00',
    'Balance': '2.00',
    'int.Br': '100'},
   {'Date': '29-07-2023',
    'Particulars': 'IMPS/P2A/321012364000/918020110872063/290',
    'Debit': '',
    'Credit': '10.00',
    'Balance': '12.00',
  

In [11]:
import json
with open('axis_bank_statement.json', 'w') as json_file:
    json.dump(axis_bank_statement, json_file, indent=4)  # 'indent' makes the JSON more readable

print("axis_bank_statement saved to axis_bank_statement.json")

axis_bank_statement saved to axis_bank_statement.json
